In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats

#Keras Referances
import tensorflow as tf
#from tensorflow.tf.keras.models import Sequential, Input,load_model
#from keras.utils.vis_utils import plot_model
#from keras.optimizers import Adam
#from keras import layers, Model, backend as K
#from keras.layers import Dense, Activation,BatchNormalization, Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dropout
#from keras.layers.normalization import BatchNormalization
#from keras.datasets import mnist



print(tf.config.experimental.list_physical_devices('GPU'))


from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPAdamOptimizer, DPGradientDescentGaussianOptimizer 


(x_train, y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
num_classes=10
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
input_shape = (x_train.shape[1], x_train.shape[1], 1)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
#data_slice = 5000
#x_train = x_train[:data_slice,:]
#y_train = y_train[:data_slice,:]
#x_test = x_test[:data_slice,:]
#y_test = y_test[:data_slice,:]


In [4]:
def Model1(input_shape=input_shape, num_classes=num_classes):    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3,3), padding=("same"), activation='relu', input_shape=x_train.shape[1:]))
    #model.add(Conv2D(32, (3,3), padding=("same"), activation='relu', input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(32,(3,3), padding=("same"), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(64,(3,3),padding=("same"),activation='relu'))
    model.add(tf.keras.layers.Conv2D(64,(3,3),padding=("same"),activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))
    return model

In [5]:
def Model2(input_shape=input_shape,num_classes=num_classes):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape, padding="same"))
    model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    model.add(tf.keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    model.add(tf.keras.layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(84, activation='relu'))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer =  tf.keras.optimizers.Adam() , loss = "categorical_crossentropy", metrics=["accuracy"])
    return model

In [6]:
#print("Model1:")
#plot_model(Model1(), to_file='Model1.png', show_shapes=True, show_layer_names=True)
#print("Model2:")
#plot_model(Model2(), to_file='Model2.png', show_shapes=True, show_layer_names=True)


In [7]:
model = []
model.append(Model1())
#model.append(Model2())

In [8]:
def plotacc(History):
   
    # summarize history for accuracy
    plt.plot(History.history['acc'])
    plt.plot(History.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(History.history['loss'])
    plt.plot(History.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [9]:
optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=1.0,
        noise_multiplier=1.1,
        num_microbatches=250,
        learning_rate=0.15)
    # Compute vector of per-example loss rather than its mean over a minibatch.
loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.losses.Reduction.NONE)



models = []
for i in range(len(model)):
    #model[i].compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
    model[i].compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
    
    #History = model[i].fit(x_train,y_train, batch_size=25, epochs=60,verbose=1,validation_data=(x_test,y_test))
    History =  model[i].fit(x_train, y_train,epochs=10,verbose=1,validation_data=(x_test, y_test),batch_size=25)
    #plotacc(History)
    models.append(model[i])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
   25/60000 [..............................] - ETA: 1:06:21

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-9-e4e9e093fbed>:19) ]] [Op:__inference_distributed_function_889]

Function call stack:
distributed_function


In [ ]:

def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=layers.average(yModels) 
    modelEns = Model(inputs=model_input, outputs=yAvg,    name='ensemble')     
    return modelEns

In [ ]:
model_input = Input(shape=models[0].input_shape[1:]) # c*h*w
modelEns = ensembleModels(models, model_input)
modelEns.summary()

In [ ]:

modelEns.save("G:\\vinay\\Models\\model1.h5")

modelEns=load_model("G:\\vinay\\Models\\model1.h5")
modelEns.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
modelEns.summary()
y=modelEns.predict(x_test)


In [ ]:
y